In [ ]:
# Reference Code : https://www.kaggle.com/seiya1998/lgbm-with-random-projection-and-aggregate-lb-1-41
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn import random_projection

import lightgbm as lgb

In [ ]:
train_df = pd.read_csv('../input/train.csv')
test_df = pd.read_csv('../input/test.csv')

X_train = train_df.drop(["ID", "target"], axis=1)
y_train = np.log1p(train_df["target"].values)

X_test = test_df.drop(["ID"], axis=1)

In [ ]:
ntrain = len(X_train)
ntest = len(X_test)

In [ ]:
print("Preparetion")
colsToRemove = []
for col in X_train.columns:
    if X_train[col].std() == 0: 
        colsToRemove.append(col)
        
# remove constant columns in the training set
X_train.drop(colsToRemove, axis=1, inplace=True)

# remove constant columns in the test set
X_test.drop(colsToRemove, axis=1, inplace=True) 

print("Removed `{}` Constant Columns\n".format(len(colsToRemove)))

In [ ]:
colsToRemove = []
colsScaned = []
dupList = {}

columns = X_train.columns

for i in range(len(columns)-1):
    v = X_train[columns[i]].values
    dupCols = []
    for j in range(i+1,len(columns)):
        if np.array_equal(v, X_train[columns[j]].values):
            colsToRemove.append(columns[j])
            if columns[j] not in colsScaned:
                dupCols.append(columns[j]) 
                colsScaned.append(columns[j])
                dupList[columns[i]] = dupCols
                
# remove duplicate columns in the training set
X_train.drop(colsToRemove, axis=1, inplace=True) 

# remove duplicate columns in the testing set
X_test.drop(colsToRemove, axis=1, inplace=True)

print("Removed `{}` Duplicate Columns\n".format(len(dupList)))

In [ ]:
print("Aggregate")
weight = ((X_train != 0).sum()/len(X_train)).values

tmp_train = X_train[X_train!=0]
tmp_test = X_test[X_test!=0]
tmp = pd.concat([X_train,X_test])#RandomProjection

X_train["weight_count"] = (tmp_train*weight).sum(axis=1)
X_test["weight_count"] = (tmp_test*weight).sum(axis=1)

X_train["count_not0"] = (X_train != 0).sum(axis=1)
X_test["count_not0"] = (X_test != 0).sum(axis=1)

X_train["sum"] = X_train.sum(axis=1)
X_test["sum"] = X_test.sum(axis=1)

X_train["var"] = tmp_train.var(axis=1)
X_test["var"] = tmp_test.var(axis=1)

X_train["mean"] = tmp_train.mean(axis=1)
X_test["mean"] = tmp_test.mean(axis=1)

X_train["std"] = tmp_train.std(axis=1)
X_test["std"] = tmp_test.std(axis=1)

X_train["max"] = tmp_train.max(axis=1)
X_test["max"] = tmp_test.max(axis=1)

X_train["min"] = tmp_train.min(axis=1)
X_test["min"] = tmp_test.min(axis=1)

del(tmp_train)
del(tmp_test)

In [ ]:
tmp.columns.value_counts().sum()

In [ ]:
n_com = 120

projection = [random_projection.SparseRandomProjection(n_components = n_com)]
transformer = projection.fit_transform(tmp)
RP = transformer.fit_transform(tmp)
# transformer = random_projection.SparseRandomProjection(n_components = n_com)
print(RP)


rp = pd.DataFrame(RP)



columns = ["RandomProjection{}".format(i) for i in range(n_com)]
rp.columns = columns

rp_train = rp[:ntrain]
rp_test = rp[ntrain:]
rp_test.index = X_test.index

#concat RandomProjection and raw data
X_train = pd.concat([X_train,rp_train],axis=1)
X_test = pd.concat([X_test,rp_test],axis=1)

In [ ]:
del(rp_train)
del(rp_test)

In [ ]:
print("Modeling")
dev_X, val_X, dev_y, val_y = train_test_split(X_train, y_train, test_size = 0.2, random_state = 42)

In [ ]:
def run_lgb(train_X, train_y, val_X, val_y, test_X):
    params = {
        "objective" : "regression",
        "metric" : "rmse",
        "num_leaves" : 64,
        "learning_rate" : 0.004,
        "bagging_fraction" : 0.7,
        "feature_fraction" : 0.5,
        "bagging_frequency" : 6,
        "bagging_seed" : 42,
        "verbosity" : 1,
        "seed": 42,
         }
    lgtrain = lgb.Dataset(train_X, label=train_y)
    lgval = lgb.Dataset(val_X, label=val_y)
    evals_result = {}
    model = lgb.train(params, lgtrain, 5000, 
                      valid_sets=[lgval], 
                      early_stopping_rounds=100, 
                      verbose_eval=50, 
                      evals_result=evals_result,
                      num_boost_round=num_boost_rounds_lgb
                      )
    pred_test_y = np.expm1(model.predict(test_X, num_iteration=model.best_iteration))
    return pred_test_y, model, evals_result

pred_test, model, evals_result = run_lgb(dev_X, dev_y, val_X, val_y, X_test)
print("LightGBM Training Completed...")


sub = pd.read_csv('../input/sample_submission.csv')
sub["target"] = pred_test
print(sub.head())
sub.to_csv('lightgbm.csv', index=False)